<a href="https://colab.research.google.com/github/hailusong/colab-god-idclass/blob/master/god_idclass_colabtrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training: Custom Train Google Object Detection to Detect ID BBox

**FIRST OF ALL: CHOOSE RUNTIME ENVIRONMENT TYPE TO BE GPU**<br>
Environment variables setup.<br>
**Tensorflow runtime version list** can be found at [here](https://cloud.google.com/ml-engine/docs/tensorflow/runtime-version-list)

In [0]:
DEFAULT_HOME='/content'
TF_RT_VERSION='1.13'
PYTHON_VERSION='3.5'

YOUR_GCS_BUCKET='id-norm'
YOUR_PROJECT='orbital-purpose-130316'

## Session and Environment Verification (Destination - Local)

Establish security session with Google Cloud

In [0]:
from google.colab import auth
auth.authenticate_user()


################# RE-RUN ABOVE CELLS IF NEED TO RESTART RUNTIME #################

Verify Versions: TF, Python, IPython and prompt_toolkit (these two need to have compatible version), and protoc

In [3]:
import tensorflow as tf
print(tf.__version__)
assert(tf.__version__.startswith(TF_RT_VERSION + '.')), f'tf.__version__ {tf.__version__} not matching with specified TF runtime version env variable {TF_RT_VERSION}'

1.13.1


In [4]:
!python -V
!ipython --version
!pip show prompt_toolkit
!protoc --version

Python 3.6.7
5.5.0
Name: prompt-toolkit
Version: 1.0.15
Summary: Library for building powerful interactive command lines in Python
Home-page: https://github.com/jonathanslenders/python-prompt-toolkit
Author: Jonathan Slenders
Author-email: UNKNOWN
License: UNKNOWN
Location: /usr/local/lib/python3.6/dist-packages
Requires: six, wcwidth
Required-by: jupyter-console, ipython
libprotoc 3.0.0


## Install Google Object Detection API in Colab
Reference is https://colab.research.google.com/drive/1kHEQK2uk35xXZ_bzMUgLkoysJIWwznYr


### Downgrade prompt-toolkit to 1.0.15 (Destination - Local)
Run this **ONLY** if the Installation not Working

In [0]:
# !pip install 'prompt-toolkit==1.0.15'

### Google Object Detection API Installation (Destination - Local)

In [6]:
!apt-get install -y -qq protobuf-compiler python-pil python-lxml
![ ! -e {DEFAULT_HOME}/models ] && git clone --depth=1 --quiet https://github.com/tensorflow/models.git {DEFAULT_HOME}/models
!ls {DEFAULT_HOME}/models

AUTHORS     CONTRIBUTING.md    LICENSE	 README.md  samples    WORKSPACE
CODEOWNERS  ISSUE_TEMPLATE.md  official  research   tutorials


In [7]:
import os
os.chdir(f'{DEFAULT_HOME}/models/research')
!pwd

/content/models/research


*From Wikipedia ...*: 

**protocol buffers** are a language-neutral, platform-neutral extensible mechanism for serializing structured data – think XML, but smaller, faster, and simpler. 

You define how you want your data to be structured once, then you can **use special generated source code to easily write and read your structured data to and from a variety of data streams and using a variety of languages**.

Remember **.proto defines structured data** and **protoc generates the source code** the serailize/de-serialize.

In [8]:
!protoc object_detection/protos/*.proto --python_out=.
# !ls object_detection/protos/*.proto
# !cat object_detection/protos/anchor_generator.proto
!ls {DEFAULT_HOME}/models/research/object_detection/builders/anchor*

/content/models/research/object_detection/builders/anchor_generator_builder.py
/content/models/research/object_detection/builders/anchor_generator_builder_test.py


#### Add Google Object Detection API into System Path

In [0]:
import sys
sys.path.append(f'{DEFAULT_HOME}/models/research')
sys.path.append(f'{DEFAULT_HOME}/models/research/slim')

Note that ! calls out to a shell (in a **NEW** process), while % affects the **SAME** process associated with the notebook.

Since we append pathes to sys.path, we **HAVE TO** use % command to run the Python

Also it is **IMPORTANT** to have **%matplotlib inline** otherwise %run model_builder_test.py will **cause function attribute error** when accessing matplotlib.pyplot attributes from **iPython's run_line_magic** 

In [0]:
# !find . -name 'inception*' -print
%matplotlib inline

In [13]:
# If see the error 'function' object has no attribute 'called', just run the %matplotlib cell and this cell AGAIN 
%run object_detection/builders/model_builder_test.py

import os
os.chdir(f'{DEFAULT_HOME}')

............s...
----------------------------------------------------------------------
Ran 16 tests in 0.117s

OK (skipped=1)


## Git Sync for any Change in colab-god-idclass 

In [14]:
![ -e {DEFAULT_HOME}/colab-god-idclass ] && git -C {DEFAULT_HOME}/colab-god-idclass pull
![ ! -e {DEFAULT_HOME}/colab-god-idclass ] && git clone --depth=1 https://github.com/hailusong/colab-god-idclass.git {DEFAULT_HOME}/colab-god-idclass

Cloning into '/content/colab-god-idclass'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14 (delta 2), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), done.


Push the latest pipeline config file to the GCS

In [15]:
!ls -al {DEFAULT_HOME}/colab-god-idclass/configs/pipeline_faster_rcnn_resnet101.config
!sed 's/..YOUR_GCS_BUCKET./{YOUR_GCS_BUCKET}/g' < {DEFAULT_HOME}/colab-god-idclass/configs/pipeline_faster_rcnn_resnet101.config > {DEFAULT_HOME}/colab-god-idclass/configs/pipeline_faster_rcnn_resnet101_processed.config
!gsutil cp {DEFAULT_HOME}/colab-god-idclass/configs/pipeline_faster_rcnn_resnet101_processed.config \
           gs://{YOUR_GCS_BUCKET}/data

-rw-r--r-- 1 root root 4320 Mar 27 13:46 /content/colab-god-idclass/configs/pipeline_faster_rcnn_resnet101.config
Copying file:///content/colab-god-idclass/configs/pipeline_faster_rcnn_resnet101_processed.config [Content-Type=application/octet-stream]...
/ [1 files][  4.2 KiB/  4.2 KiB]                                                
Operation completed over 1 objects/4.2 KiB.                                      


### Checking Your Google Cloud Storage Bucket

In [16]:
!gsutil ls gs://{YOUR_GCS_BUCKET}/data
!gsutil ls gs://{YOUR_GCS_BUCKET}/generated

gs://id-norm/data/faster_rcnn_resnet101_processed.config
gs://id-norm/data/label_map.pbtxt
gs://id-norm/data/model.ckpt.data-00000-of-00001
gs://id-norm/data/model.ckpt.index
gs://id-norm/data/model.ckpt.meta
gs://id-norm/data/pipeline_faster_rcnn_resnet101_processed.config
gs://id-norm/data/test.record
gs://id-norm/data/train.record
gs://id-norm/generated/bbox-train-non-id1.csv
gs://id-norm/generated/bbox-train-non-id2.csv
gs://id-norm/generated/bbox-train-non-id3.csv
gs://id-norm/generated/bbox-train-on-dl.csv
gs://id-norm/generated/bbox-train-on-hc.csv
gs://id-norm/generated/bbox-valid-non-id1.csv
gs://id-norm/generated/bbox-valid-non-id2.csv
gs://id-norm/generated/bbox-valid-non-id3.csv
gs://id-norm/generated/bbox-valid-on-dl.csv
gs://id-norm/generated/bbox-valid-on-hc.csv
gs://id-norm/generated/pnts-train-non-id1.csv
gs://id-norm/generated/pnts-train-non-id2.csv
gs://id-norm/generated/pnts-train-non-id3.csv
gs://id-norm/generated/pnts-train-on-dl.csv
gs://id-norm/generated/pnts-tr

## Start the Training and Evaluation Jobs on Google Cloud ML Engine

### Option 2: Start the Training Job on CoLab

**Clean up the those tf.app.flags usde by Object Detection API** (so that we don't need to restart the runtime if we want to %run the Google object detection API again in the same session)

In [0]:
def del_all_flags(FLAGS, excls):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        if keys in excls:
          print(f'SKIPPING exclusion attribute {keys}')
          continue
          
        print(f'removing attribute {keys}')
        FLAGS.__delattr__(keys)


# if running inside IPython notebook, the python session will be maintained across
# cells, so does the tf.app.flags. That will cause flags defined twice error
# if we %run the app multiple times. The workaroud is to always clean up
# the flags before defining them.
# --------------------------
# flags = tf.app.flags
# del_all_flags(flags.FLAGS, ['logtostderr'])
# --------------------------

# flags.DEFINE_string('logtostderr', '', '')


MAKE SURE YOU SET RUNTIME TYPE TO **GPU or TPU**

In [0]:
import os
os.chdir(f'{DEFAULT_HOME}/models/research')

import sys
sys.path.append(f'{DEFAULT_HOME}/models/research')
sys.path.append(f'{DEFAULT_HOME}/models/research/slim')

# Start the training
%run object_detection/model_main.py \
     --logtostderr \
     --model_dir=gs://{YOUR_GCS_BUCKET}/model_dir \
     --pipeline_config_path=gs://{YOUR_GCS_BUCKET}/data/pipeline_faster_rcnn_resnet101_processed.config

W0327 13:46:23.424210 139747382667136 model_lib.py:598] Forced number of epochs for all eval validations to be 1.


INFO:tensorflow:Maybe overwriting train_steps: None


I0327 13:46:23.429358 139747382667136 config_util.py:482] Maybe overwriting train_steps: None


INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1


I0327 13:46:23.431100 139747382667136 config_util.py:482] Maybe overwriting sample_1_of_n_eval_examples: 1


...

I0327 13:46:30.671915 139747382667136 regularizers.py:98] Scale of 0 disables regularizer.


INFO:tensorflow:Scale of 0 disables regularizer.


I0327 13:46:30.703719 139747382667136 regularizers.py:98] Scale of 0 disables regularizer.


INFO:tensorflow:depth of additional conv before box predictor: 0


I0327 13:46:30.707051 139747382667136 convolutional_box_predictor.py:150] depth of additional conv before box predictor: 0


### Option 2 - Monitor CoLab Training using Tensorboard running on Colab
**OBVIOUSLY YOU CANNOT BOTH TRAIN and MONITOR on COLAB AT THE SAME TIME. ONE SESSION WILL BE STOPED**<br>
You will need to install ngrok for tunneling purpose

In [0]:
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip

In [0]:
# get_ipython().system_raw('./ngrok http 6006 &')
# !curl -s http://localhost:4040/api/tunnels | python3 -c \
   "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Start the local Tensorboard with log data feed from your GCS bucket and then **click on the link above**

First, we need to do the authorization

In [0]:
# This command needs to be run once to allow your local machine to access your
# GCS bucket.
# !gcloud auth application-default login

Now time to start the **Tensorboard**

In [0]:
# !tensorboard --logdir=gs://{YOUR_GCS_BUCKET}/model_dir

### Option 3 - Monitor CoLab Training Job using Tensorboard on Google Cloud Shell
* Log into the Google Cloud and run cloud shell
* In the shell run
```
export YOUR_GCS_BUCKET='id-norm'
tensorboard --logdir=gs://$YOUR_GCS_BUCKET/model_dir
```
* Web preview on tensorboard port (e.g. 6006)